In [ ]:
import torch
import sys
sys.path.append('../datasets/')
sys.path.append('../models/')
from models.rec_vae import VAE_Module
from datasets.rec_fmnist import RecFMNIST
from argparse import ArgumentParser, Namespace
import os
from torch.utils.data import DataLoader
from torch.functional import F
import matplotlib.pyplot as plt
os.chdir('/mnt/data/Dottorato/canomaly/.')

In [ ]:
reconstruction_loss = lambda rec, x:  F.mse_loss(rec, x, reduction='none').mean(dim=[i for i in range(len(rec.shape))][1:]) 

In [ ]:
args = Namespace(approach='continual',
                add_rotation=False,
                batch_size=1000)

In [ ]:
# %tb
# parser = ArgumentParser()
# args = parser.parse_known_args()
dataset = RecFMNIST(args)
test_dataset = dataset.test_loader()


In [ ]:
models_list = os.listdir(dataset.config.checkpoints_dir)

In [ ]:
model0 = torch.load(os.path.join(dataset.config.checkpoints_dir, [x for x in models_list if '0' in x][0]))
model3 = torch.load(os.path.join(dataset.config.checkpoints_dir, [x for x in models_list if '3' in x][0]))
model0.eval()
model3.eval()

In [ ]:
X,Y = next(iter(test_dataset))
X,Y = X.to('cuda'),Y.to('cuda')

In [ ]:
model3.forward_sample = True
recs, latent_mu, latent_logvar, z  = model3(X)
recs_mist = model0.D(z)

In [ ]:
real_loss = reconstruction_loss(recs, X)
mist_loss = reconstruction_loss(recs_mist, X)

knowledge = dataset.MACRO_CLASSES

targets_knowledge = Y.clone()
for num, kn in enumerate(knowledge):
    targets_knowledge[torch.isin(Y, torch.Tensor(kn).to('cuda'))] = num    

In [ ]:
print(real_loss[targets_knowledge==0].mean())
print(mist_loss[targets_knowledge==0].mean())

In [ ]:
recs, latent_mu, latent_logvar, z  = model0(X)
recs_mist = model3.D(latent_mu)

real_loss = reconstruction_loss(recs, X)
mist_loss = reconstruction_loss(recs_mist, X)

knowledge = dataset.MACRO_CLASSES

targets_knowledge = Y.clone()
for num, kn in enumerate(knowledge):
    targets_knowledge[torch.isin(Y, torch.Tensor(kn).to('cuda'))] = num    

In [ ]:
print(real_loss[targets_knowledge==0].mean())
print(mist_loss[targets_knowledge==0].mean())

In [ ]:
index_zeros = targets_knowledge == 0
index_zeros = index_zeros.nonzero(as_tuple=True)[0]
for i in index_zeros[:50]:
    rec_m = recs_mist[i]
    rec = recs[i]
    real = X[i]
    fig,ax = plt.subplots(1,3)
    ax[0].imshow(rec_m.permute(1,2,0).detach().cpu())
    ax[1].imshow(rec.permute(1,2,0).detach().cpu())
    ax[2].imshow(real.permute(1,2,0).detach().cpu())
    plt.show()